### ----前置作業 下載----
#### 負責人09156248 黃登祥 

In [11]:
!pip install pymssql # 連線SQL需安裝下載套件

Active code page: 936
     ---------------------------------------- 1.5/1.5 MB 3.6 MB/s eta 0:00:00


In [12]:
!pip install pyodbc # ODBC的DRIVER(驅動程式)

Active code page: 936


### ----開始執行----

In [2]:
# coding=utf-8
from platform import python_version
import os, time, socket, glob, pickle, joblib
import pandas as pd
import pyodbc, pymssql
import warnings
warnings.filterwarnings(action="ignore")
print("【現在時間】{}".format(time.strftime("%Y/%m/%d %H:%M:%S")))
print("【所在目錄】{}".format(os.getcwd()))
#print("【主機名稱】{} ({})".format(socket.gethostname(),socket.gethostbyname(socket.
print("【Python】{}".format(python_version()))
print("【pymssql】{}".format(pymssql.__version__))
print("【pyodbc】{}".format(pyodbc.version))
%autosave 120

【現在時間】2022/12/16 23:55:23
【所在目錄】C:\Users\User\Desktop\資料工程
【Python】3.9.13
【pymssql】2.2.7
【pyodbc】4.0.34


Autosaving every 120 seconds


In [3]:
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup

### ----更改股票代號----

In [4]:
stock_id = "6446" 
url = "https://goodinfo.tw/tw/StockDetail.asp?STOCK_ID=" + stock_id
payloads = {"STOCK_ID":stock_id}
headers = \
{"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}

In [5]:
url = "https://goodinfo.tw/tw/EquityDistributionClassHis.asp?STOCK_ID="+stock_id
resp = requests.request(method="POST",url=url,data=payloads,headers=headers)
resp.encoding = "utf-8"

equityDclass_table = pd.read_html(resp.text)

In [ ]:
df = equityDclass_table[18].dropna(axis='columns')
df

### ----將DataFrame 轉成list----

In [150]:
products_list = df.values.tolist()
print(products_list[59])
del products_list[159]
del products_list[158]
del products_list[139]
del products_list[138]
del products_list[119]
del products_list[118]
del products_list[99]
del products_list[98]
del products_list[79]
del products_list[78]
del products_list[59]
del products_list[58]
del products_list[39]
del products_list[38]
del products_list[19]
del products_list[18]
lists_num=len(products_list)
print(len(products_list))
print(products_list)

['週別', '統計日期', '收盤', '漲跌(元)', '漲跌(%)', '集保庫存(萬張)', '≦10張', '＞10張≦50張', '＞50張≦100張', '＞100張≦200張', '＞200張≦400張', '＞400張≦800張', '＞800張≦1千張', '＞1千張']
156
[['22W51', '-', '525', '-12', '-2.23', '-', '-', '-', '-', '-', '-', '-', '-', '-'], ['22W50', '12/09', '537', '+5', '+0.94', '30.27', '9.91', '12.9', '6.7', '8.04', '8.25', '7.08', '2', '45.1'], ['22W49', '12/02', '532', '+26', '+5.14', '30.27', '10', '12.8', '6.74', '8.19', '8.32', '6.61', '2.01', '45.3'], ['22W48', '11/25', '506', '-22', '-4.17', '30.27', '9.93', '12.5', '6.48', '7.84', '6.94', '6.41', '0.84', '49'], ['22W47', '11/18', '528', '+20', '+3.94', '30.27', '10.1', '12.6', '6.52', '7.6', '7.15', '6.43', '1.15', '48.5'], ['22W46', '11/11', '508', '+5', '+0.99', '30.27', '10.2', '12.7', '6.58', '7.51', '6.82', '6.94', '0.84', '48.5'], ['22W45', '11/04', '503', '+55', '+12.28', '30.27', '9.94', '12.5', '6.74', '7.39', '6.7', '7.26', '1.21', '48.2'], ['22W44', '10/28', '448', '0', '0', '30.27', '10.1', '12.6', '6.85', '7.37', '6

'''0         1     2     3      4      5    6  7  8  9   10   11  12 '''
'''周別    日期    收盤  漲      跌  集保庫存 10 50 100 200 400 800 1000  '''


### SQL連線

In [132]:
connsqlodbc= pyodbc.connect("DRIVER={SQL Server Native Client 11.0};SERVER=124.218.194.84;"
                      "DATABASE=testDB;UID=testuser;PWD=323")


### 創表

In [100]:
connsqlodbc.autocommit = True
cursor = connsqlodbc.cursor()
# GS+代號+發周別
# MYSQL指令


sqlStr ='''create table 持股等級股東持有比例表
(
	持股等級股東持有比例表編號 nvarchar(20) primary key
	,[股東持有張數比例(≦10張)] nvarchar(20) NULL
	,[股東持有張數比例(>10張&≦50張)] nvarchar(20) Null
	,[股東持有張數比例(>50張&≦100張)] nvarchar(20) Null
	,[股東持有張數比例(＞100張≦200張)] nvarchar(20) Null
	,[股東持有張數比例(＞200張≦400張)] nvarchar(20) Null
	,[股東持有張數比例(＞400張≦800張)] nvarchar(20) Null
	,[股東持有張數比例(＞800張≦1千張)] nvarchar(20) Null
	,[股東持有張數比例(＞1千張)] nvarchar(20) Null
	,[集保庫存(萬張)]nvarchar(10) Null
	,統計日期 nvarchar(5) not Null
	,持股等級周別 text not Null
	,股票代號 nvarchar(10) not Null
)
'''
cursor.execute(sqlStr)

### 資料入庫

In [146]:
connsqlodbc.autocommit = True

connsqlodbc.setdecoding(pyodbc.SQL_CHAR, encoding='utf-8')
connsqlodbc.setencoding('utf-8')
cursor = connsqlodbc.cursor()
# GS+代號+發周別
# MYSQL指令
for i in range(0,lists_num):
    Gstock_id='GS'+str(stock_id)+str(products_list[i][0])
    sqlStr = f""" 
    Insert into 持股等級股東持有比例表 (持股等級股東持有比例表編號, 
    [股東持有張數比例(≦10張)], [股東持有張數比例(>10張&≦50張)],
    [股東持有張數比例(>50張&≦100張)],[股東持有張數比例(＞100張≦200張)],
    [股東持有張數比例(＞200張≦400張)],[股東持有張數比例(＞400張≦800張)],
    [股東持有張數比例(＞800張≦1千張)],[股東持有張數比例(＞1千張)],
    [集保庫存(萬張)],統計日期,持股等級周別,股票代號)
    values ('{str(Gstock_id)}',
    '{products_list[i][6]}','{products_list[i][7]}','{products_list[i][8]}','{products_list[i][9]}',
    '{products_list[i][10]}','{products_list[i][11]}','{products_list[i][12]}','{products_list[i][13]}',
    '{products_list[i][5]}','{products_list[i][1]}','{products_list[i][0]}','{stock_id}')
    
    """
    cursor.execute(sqlStr)

print(sqlStr)

 
    Insert into 持股等級股東持有比例表 (持股等級股東持有比例表編號, 
    [股東持有張數比例(≦10張)], [股東持有張數比例(>10張&≦50張)],
    [股東持有張數比例(>50張&≦100張)],[股東持有張數比例(＞100張≦200張)],
    [股東持有張數比例(＞200張≦400張)],[股東持有張數比例(＞400張≦800張)],
    [股東持有張數比例(＞800張≦1千張)],[股東持有張數比例(＞1千張)],
    [集保庫存(萬張)],統計日期,持股等級周別,股票代號)
    values ('GS644619W51',
    '9.54','12.6','6.67','6.07',
    '6.66','6.26','1.99','50.2',
    '21.94','12/20','19W51','6446')
    
    


In [147]:
conn.close()

In [ ]:
'''
MySQL 持股等級股東持有比例表 格式

GS+代號+發周別 
比例
集保庫存
日期
周別
代號


''' 
# MySQL組合鍵
# https://www.delftstack.com/zh-tw/howto/mysql/introduction-to-the-mysql-composite-key/
